In [ ]:
!pip install turicreate

     |████████████████████████████████| 92.0MB 57kB/s 
     |████████████████████████████████| 327kB 39.3MB/s 
     |████████████████████████████████| 3.6MB 27.1MB/s 
     |████████████████████████████████| 3.5MB 17.0MB/s 
     |████████████████████████████████| 86.4MB 55kB/s 
     |████████████████████████████████| 18.3MB 251kB/s 
     |████████████████████████████████| 3.8MB 27.0MB/s 
     |████████████████████████████████| 2.9MB 32.3MB/s 
     |████████████████████████████████| 450kB 49.9MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp37-none-any.whl size=320858 sha256=28211656b729c0f3bb64f6c4a0ab46c4e3ea361c8732dc7a8e455810b564dc59
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
  Created wheel for prettytable: filename=prettytable-0.7.2-cp37-none-any.whl size=13716 sha256=8ee9a0b0d917a0ba9235754a2fbfd5d6fe0e33a847ed5d5128e25802a91d594b
  Stored in directory:

In [ ]:
import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split
# import scripts as data_layer
import sys
sys.path.append("..")

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
customers = pd.read_csv('/content/drive/MyDrive/Item Based Collaborative/USD/USDCustDesc.csv')
rekomendasi = pd.read_csv('/content/drive/MyDrive/Item Based Collaborative/USD/rekomen.csv')

In [ ]:
rekomendasi.head

<bound method NDFrame.head of      customer_id
0         100003
1         100004
2         100012
3         100017
4         100018
..           ...
364       100209
365       100255
366       100688
367       100905
368       100907

[369 rows x 1 columns]>

In [ ]:
customers.head

<bound method NDFrame.head of        customer_id      customer_name  id_product  ...   amount bulan tahun
0           100003  ABADI JAYA 2 TOKO      100918  ...  2590909    12  2020
1           100003  ABADI JAYA 2 TOKO      101112  ...   229091    12  2020
2           100003  ABADI JAYA 2 TOKO      101121  ...   134000    12  2020
3           100003  ABADI JAYA 2 TOKO      101133  ...   229091    12  2020
4           100004    ABADI JAYA TOKO      100077  ...   723273    12  2020
...            ...                ...         ...  ...      ...   ...   ...
10946       100907            DEPO QU      101024  ...   695455     5  2021
10947       100907            DEPO QU      101025  ...   695454     5  2021
10948       100907            DEPO QU      101061  ...  1069091     5  2021
10949       100907            DEPO QU      101063  ...  1069091     5  2021
10950       100907            DEPO QU      101076  ...  3636364     5  2021

[10951 rows x 10 columns]>

In [ ]:
data = customers.groupby(['customer_id', 'id_product'])["qty"].sum()

In [ ]:
print(data.shape)
data.head()

(5368,)


customer_id  id_product
100002       101106          7
             101110         15
100003       100629         54
             100643        141
             100871         60
Name: qty, dtype: int64

In [ ]:
data = customers[["customer_id","id_product","qty"]].dropna()

In [ ]:
data.head()

,customer_id,id_product,qty
0,100003,100918,50
1,100003,101112,3
2,100003,101121,2
3,100003,101133,3
4,100004,100077,12


In [ ]:
data['customer_id'] = data['customer_id'].astype(np.int64)
data['qty'] = data['qty'].astype(np.int64)

In [ ]:
data.dtypes

customer_id    int64
id_product     int64
qty            int64
dtype: object

In [ ]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [ ]:
data_dummy = create_data_dummy(data)

In [ ]:
df_matrix = pd.pivot_table(data, values='qty', index='customer_id', columns='id_product')
df_matrix.head()

id_product,100006,100015,100016,100019,100032,100041,100044,100057,100076,100077,100078,100083,100092,100093,100094,100096,100097,100098,100101,100110,100112,100113,100117,100122,100123,100175,100288,100312,100323,100324,100328,100340,100341,100343,100346,100348,100353,100356,100372,100487,...,101148,101149,101150,101151,101152,101153,101155,101156,101157,101158,101159,101160,101161,101162,101163,101164,101165,101166,101167,101168,101169,101170,101171,101172,101173,101175,101176,101177,101178,101179,101180,101181,101182,101184,101185,101186,101187,101190,101191,101196
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,23.0,48.5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,10.0,7.5,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.333333,NaN,NaN,NaN,55.0,NaN,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
(df_matrix.shape)

(369, 303)

In [ ]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(369, 303)


id_product,100006,100015,100016,100019,100032,100041,100044,100057,100076,100077,100078,100083,100092,100093,100094,100096,100097,100098,100101,100110,100112,100113,100117,100122,100123,100175,100288,100312,100323,100324,100328,100340,100341,100343,100346,100348,100353,100356,100372,100487,...,101148,101149,101150,101151,101152,101153,101155,101156,101157,101158,101159,101160,101161,101162,101163,101164,101165,101166,101167,101168,101169,101170,101171,101172,101173,101175,101176,101177,101178,101179,101180,101181,101182,101184,101185,101186,101187,101190,101191,101196
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.218274,0.477157,0.015075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.035533,0.086294,0.032663,NaN,0.242424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.566667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.814815,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.018613,NaN,NaN,NaN,0.543147,NaN,0.130653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.010381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customer_id'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(5329, 3)


,customer_id,id_product,scaled_purchase_freq
29,100100,100006,0.104167
42,100155,100006,0.093750
43,100157,100006,0.010417
54,100193,100006,0.003472
59,100219,100006,1.000000


In [ ]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='qty', index='customer_id', columns='id_product')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customer_id'], value_name='scaled_purchase_freq').dropna()

In [ ]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(8760, 3) (2191, 3)


In [ ]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [ ]:
train_data

customer_id,id_product,qty
100479,101013,20
100522,101029,4
100479,100862,16
100478,100044,6
100531,101140,50
100799,101125,4
100809,100643,250
100657,101148,5
100833,101128,13
100806,100886,18


In [ ]:
test_data

customer_id,id_product,qty
100484,101066,11
100567,100988,18
100803,100871,100
100479,100851,3
100254,101094,3
100799,101112,14
100155,100937,60
100665,101060,1
100625,101099,16
100542,100083,2


In [ ]:
# We can define a function for this step as follows

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [ ]:
# lets try with both dummy table and scaled/normalized purchase table

train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

Using purchase counts

In [ ]:
# variables to define field names
user_id = 'customer_id'
item_id = 'id_product'
target = 'qty'
users_to_recommend = list(rekomendasi[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [ ]:
popularity_model = tc.popularity_recommender.create(train_data, user_id=user_id, item_id=item_id, target=target)

Preparing data set.

Data has 8760 observations with 361 users and 292 items.

Data prepared in: 0.020658s

8760 observations to process; with 292 unique items.

In [ ]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

+-------------+------------+--------------------+------+
| customer_id | id_product |       score        | rank |
+-------------+------------+--------------------+------+
|    100003   |   101196   | 533.3333333333334  |  1   |
|    100003   |   100288   | 329.8936170212766  |  2   |
|    100003   |   101181   |       125.0        |  3   |
|    100003   |   101066   | 119.88947368421053 |  4   |
|    100003   |   101091   | 114.14285714285714 |  5   |
|    100003   |   101191   | 105.33333333333333 |  6   |
|    100003   |   101190   | 101.78571428571429 |  7   |
|    100003   |   100778   |       100.0        |  8   |
|    100003   |   100117   |       100.0        |  9   |
|    100003   |   101159   | 94.69444444444444  |  10  |
|    100004   |   101196   | 533.3333333333334  |  1   |
|    100004   |   100288   | 329.8936170212766  |  2   |
|    100004   |   101076   |       136.0        |  3   |
|    100004   |   101181   |       125.0        |  4   |
|    100004   |   101066   | 11


Define a model function for model selection

In [ ]:
# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [ ]:
# variables to define field names
# constant variables include:
user_id = 'customer_id'
item_id = 'id_product'
users_to_recommend = list(rekomendasi[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

Using purchase dummy

In [ ]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns qty;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 8760 observations with 365 users and 297 items.

Data prepared in: 0.048755s

8760 observations to process; with 297 unique items.

+-------------+------------+-------+------+
| customer_id | id_product | score | rank |
+-------------+------------+-------+------+
|    100003   |   101151   |  1.0  |  1   |
|    100003   |   100877   |  1.0  |  2   |
|    100003   |   101086   |  1.0  |  3   |
|    100003   |   101017   |  1.0  |  4   |
|    100003   |   100891   |  1.0  |  5   |
|    100003   |   100627   |  1.0  |  6   |
|    100003   |   100348   |  1.0  |  7   |
|    100003   |   101130   |  1.0  |  8   |
|    100003   |   100915   |  1.0  |  9   |
|    100003   |   101066   |  1.0  |  10  |
|    100004   |   101151   |  1.0  |  1   |
|    100004   |   100877   |  1.0  |  2   |
|    100004   |   101086   |  1.0  |  3   |
|    100004   |   101017   |  1.0  |  4   |
|    100004   |   100891   |  1.0  |  5   |
|    100004   |   100627   |  1.0  |  6   |
|    100004   |   100348   |  1.0  |  7   |
|    100004   |   101130   |  1.0  |  8   |
|    100004   |   100915   |  1.0  |  9   |
|    100004   |   101066   |  1.

Using normalized purchase count

In [ ]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 4263 observations with 361 users and 266 items.

Data prepared in: 0.02218s

4263 observations to process; with 266 unique items.

+-------------+------------+--------------------+------+
| customer_id | id_product |       score        | rank |
+-------------+------------+--------------------+------+
|    100003   |   101181   |        1.0         |  1   |
|    100003   |   100571   |        1.0         |  2   |
|    100003   |   101043   | 0.8471074380165289 |  3   |
|    100003   |   100032   | 0.6629213483146067 |  4   |
|    100003   |   100891   | 0.6527395295467585 |  5   |
|    100003   |   100914   | 0.6526315789473685 |  6   |
|    100003   |   100077   |       0.625        |  7   |
|    100003   |   101170   | 0.6079622132253711 |  8   |
|    100003   |   100122   | 0.6049382716049383 |  9   |
|    100003   |   100112   | 0.5941734417344173 |  10  |
|    100004   |   101181   |        1.0         |  1   |
|    100004   |   100571   |        1.0         |  2   |
|    100004   |   101043   | 0.8471074380165289 |  3   |
|    100004   |   100032   | 0.6629213483146067 |  4   |
|    100004   |   100891   | 0.

In [ ]:
train.groupby(by=item_id)['qty'].mean().sort_values(ascending=False).head(20)

id_product
101196    533.333333
100288    329.893617
101076    136.000000
101181    125.000000
101066    119.889474
101091    114.142857
101191    105.333333
101190    101.785714
100117    100.000000
100778    100.000000
100871     97.229391
101159     94.694444
100953     94.415094
101161     93.000000
100640     92.428571
101160     92.270270
101185     91.523810
101173     90.272727
100627     89.780000
100933     88.176471
Name: qty, dtype: float64

# Collaborative Filtering Model

Using purchase count

In [ ]:
# these variables will change accordingly
name = 'cosine'
target = 'qty'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 8760 observations with 361 users and 292 items.

Data prepared in: 0.024623s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.017092s

+-------------+------------+--------------------+------+
| customer_id | id_product |       score        | rank |
+-------------+------------+--------------------+------+
|    100003   |   101140   |  5.47059353813529  |  1   |
|    100003   |   101163   | 4.9243682362139225 |  2   |
|    100003   |   100962   | 4.876722797751427  |  3   |
|    100003   |   101175   | 4.830155566334724  |  4   |
|    100003   |   100701   | 4.8280812203884125 |  5   |
|    100003   |   100851   | 4.826858300715685  |  6   |
|    100003   |   101150   | 4.745644811540842  |  7   |
|    100003   |   101181   | 4.729232657700777  |  8   |
|    100003   |   101130   |  4.64811684191227  |  9   |
|    100003   |   100907   | 4.483646187931299  |  10  |
|    100004   |   101163   | 5.075075473388036  |  1   |
|    100004   |   101150   | 4.535204575459162  |  2   |
|    100004   |   100701   | 4.493114997943242  |  3   |
|    100004   |   101181   |  4.45920774936676  |  4   |
|    100004   |   101175   | 4.

Using purchase dummy

In [ ]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns qty;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 8760 observations with 365 users and 297 items.

Data prepared in: 0.02068s

...

| 12.765ms                            | 100              | 297             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.020818s

Using normalized purchase count

In [ ]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 4263 observations with 361 users and 266 items.

Data prepared in: 0.02462s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

+-------------+------------+----------------------+------+
| customer_id | id_product |        score         | rank |
+-------------+------------+----------------------+------+
|    100003   |   101181   | 0.03001156250635783  |  1   |
|    100003   |   100701   | 0.029650616645812988 |  2   |
|    100003   |   101163   | 0.029622324307759604 |  3   |
|    100003   |   101150   | 0.029281842708587646 |  4   |
|    100003   |   100962   | 0.02909404436747233  |  5   |
|    100003   |   100851   | 0.028416721026102702 |  6   |
|    100003   |   101000   | 0.02682200272878011  |  7   |
|    100003   |   101051   | 0.025971639156341552 |  8   |
|    100003   |   101178   | 0.025065751870473225 |  9   |
|    100003   |   101175   | 0.023204068342844646 |  10  |
|    100004   |   101163   | 0.042652266025543215 |  1   |
|    100004   |   101148   | 0.04027101755142212  |  2   |
|    100004   |   101137   | 0.03734103918075562  |  3   |
|    100004   |   101000   | 0.03688416957855225  |  4  

Finished training in 0.019371s

**Pearson similarity**

In [ ]:
# these variables will change accordingly
name = 'pearson'
target = 'qty'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 8760 observations with 361 users and 292 items.

Data prepared in: 0.023045s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 16.678ms                            | 100              | 292             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.022123s

Using purchase dummy

In [ ]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns qty;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 8760 observations with 365 users and 297 items.

Data prepared in: 0.033434s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

+-------------+------------+-------+------+
| customer_id | id_product | score | rank |
+-------------+------------+-------+------+
|    100003   |   101151   |  0.0  |  1   |
|    100003   |   100877   |  0.0  |  2   |
|    100003   |   101086   |  0.0  |  3   |
|    100003   |   101017   |  0.0  |  4   |
|    100003   |   100891   |  0.0  |  5   |
|    100003   |   100627   |  0.0  |  6   |
|    100003   |   100348   |  0.0  |  7   |
|    100003   |   101130   |  0.0  |  8   |
|    100003   |   100915   |  0.0  |  9   |
|    100003   |   101066   |  0.0  |  10  |
|    100004   |   101151   |  0.0  |  1   |
|    100004   |   100877   |  0.0  |  2   |
|    100004   |   101086   |  0.0  |  3   |
|    100004   |   101017   |  0.0  |  4   |
|    100004   |   100891   |  0.0  |  5   |
|    100004   |   100627   |  0.0  |  6   |
|    100004   |   100348   |  0.0  |  7   |
|    100004   |   101130   |  0.0  |  8   |
|    100004   |   100915   |  0.0  |  9   |
|    100004   |   101066   |  0.

Generating candidate set for working with new users.

Finished training in 0.037229s

Using normalized purchase count

In [ ]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 4263 observations with 361 users and 266 items.

Data prepared in: 0.01598s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.020218s

+-------------+------------+--------------------+------+
| customer_id | id_product |       score        | rank |
+-------------+------------+--------------------+------+
|    100003   |   101181   |        1.0         |  1   |
|    100003   |   100571   |        1.0         |  2   |
|    100003   |   101043   | 0.8467445490445643 |  3   |
|    100003   |   100032   | 0.6615564536066091 |  4   |
|    100003   |   100914   | 0.6526315789473685 |  5   |
|    100003   |   100891   | 0.6513769037889051 |  6   |
|    100003   |   100077   | 0.6245579481124878 |  7   |
|    100003   |   101170   | 0.6073728981288338 |  8   |
|    100003   |   100122   | 0.6047197212407619 |  9   |
|    100003   |   100112   | 0.5941734417344174 |  10  |
|    100004   |   101181   |        1.0         |  1   |
|    100004   |   100571   |        1.0         |  2   |
|    100004   |   101043   | 0.8467678918129156 |  3   |
|    100004   |   100032   | 0.6599570185950633 |  4   |
|    100004   |   100891   | 0.

**Model Evaluation**

In [ ]:
# create initial callable variables

models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']


Models on purchase counts

In [ ]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts

Precision and recall summary statistics by cutoff
+--------+----------------------+-----------------------+
| cutoff |    mean_precision    |      mean_recall      |
+--------+----------------------+-----------------------+
|   1    |         0.0          |          0.0          |
|   2    | 0.003571428571428572 | 0.0005521048999309868 |
|   3    | 0.008333333333333331 |  0.010968771566597655 |
|   4    | 0.006250000000000001 |  0.010968771566597655 |
|   5    | 0.013571428571428576 |  0.024988941037894945 |
|   6    | 0.013095238095238092 |   0.0258055962023762  |
|   7    | 0.011224489795918365 |   0.0258055962023762  |
|   8    | 0.010714285714285716 |  0.02665593633843061  |
|   9    | 0.010317460317460319 |  0.02745950776700205  |
|   10   | 0.011071428571428578 |  0.028562968482531727 |
+--------+----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 96.32241608221572

Per User RMSE (best

Models on purchase dummy

In [ ]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy

Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.003401360544217688 | 9.448223733938017e-05 |
|   2    |  0.010204081632653064 | 0.0011392572711165906 |
|   3    |  0.009070294784580499 | 0.0017061506951528715 |
|   4    |  0.007653061224489796 |  0.002131320763180082 |
|   5    |  0.007482993197278912 |  0.002934419780564814 |
|   6    |  0.007369614512471656 |  0.003016145834830914 |
|   7    |  0.007774538386783284 | 0.0038725598290000117 |
|   8    | 0.0076530612244897975 |  0.004892967992265318 |
|   9    |  0.008314436885865459 | 0.0063042157211705445 |
|   10   |  0.007823129251700684 |  0.006362859868484641 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.0

Per User RMSE (best)

Models on normalized purchase frequency

In [ ]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts

Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.007352941176470588 |  0.001045892494929006 |
|   2    | 0.0055147058823529415 | 0.0013801170938594874 |
|   3    |  0.003676470588235294 | 0.0013801170938594874 |
|   4    |  0.003676470588235295 | 0.0015162826712015348 |
|   5    |  0.00588235294117647  |  0.006841329356542338 |
|   6    |  0.004901960784313726 |  0.006841329356542338 |
|   7    | 0.0057773109243697465 |  0.009320161798610074 |
|   8    |  0.006893382352941176 |  0.016196523454383494 |
|   9    |  0.006944444444444447 |  0.01647371766540124  |
|   10   |  0.00661764705882353  |  0.016841364724224768 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.285395738337351

Final Output

In [ ]:
users_to_recommend = list(rekomendasi[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns qty;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 10951 observations with 369 users and 303 items.

Data prepared in: 0.02714s

...

+-------------------------------------+------------------+-----------------+

+-------------+------------+---------------------+------+
| customer_id | id_product |        score        | rank |
+-------------+------------+---------------------+------+
|    100003   |   101015   | 0.26418859818402457 |  1   |
|    100003   |   101110   |  0.2641370366601383 |  2   |
|    100003   |   100640   | 0.24680976657306447 |  3   |
|    100003   |   100917   | 0.24383309308220358 |  4   |
|    100003   |   100832   | 0.23385548942229328 |  5   |
|    100003   |   100904   | 0.22453951835632324 |  6   |
|    100003   |   100753   | 0.21760079439948588 |  7   |
|    100003   |   101077   | 0.21066128856995525 |  8   |
|    100003   |   100956   | 0.20984654216205373 |  9   |
|    100003   |   101028   |  0.2072528776000528 |  10  |
|    100004   |   100917   |  0.2145597618073225 |  1   |
|    100004   |   100956   | 0.21380746737122536 |  2   |
|    100004   |   101015   | 0.21127400733530521 |  3   |
|    100004   |   100753   | 0.20499004796147346 |  4   |
|    100004   

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.043571s

In [ ]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(3690, 4)


,customer_id,id_product,score,rank
0,100003,101015,0.264189,1
1,100003,101110,0.264137,2
2,100003,100640,0.246810,3
3,100003,100917,0.243833,4
4,100003,100832,0.233855,5


In [ ]:
def create_output(model, users_to_recommend, n_rec, print_json=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customer_id', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customer_id').set_index('customer_id')
    if print_json:
        df_output.to_json('/content/drive/MyDrive/Item Based Collaborative/STI/collaborative_filtering.json')
        print("An output file can be found in 'output' folder with name 'collaborative_filtering.json'")
    return df_output

In [ ]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customer_id', 'recommendedProducts']].drop_duplicates().sort_values('customer_id').set_index('customer_id')

In [ ]:
df_output = create_output(final_model, users_to_recommend, n_rec, print_json=True)
print(df_output.shape)
df_output.head()

An output file can be found in 'output' folder with name 'collaborative_filtering.json'
(369, 1)


,recommendedProducts
customer_id,
100002,100988|101107|100643|101147|100941|100832|1010...
100003,101015|101110|100640|100917|100832|100904|1007...
100004,100917|100956|101015|100753|100543|100979|1010...
100012,100640|100643|100904|100915|101130|100889|1009...
100017,100956|100832|100753|100918|100640|100917|1010...


Customer recommendation function

In [ ]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [ ]:
customer_recomendation(100026)

recommendedProducts    111454|111450|101298|111035|111438|111458|1114...
Name: 100026, dtype: object